# Einfluss von Temperatur auf Verbrauch 

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

In [ ]:
df = pd.read_csv(r"Dateipfad\Ladevorgänge_H_R_SZ.csv",sep=';',encoding='latin-1')
df2 = pd.read_csv("Temperatur_mittel.csv",sep='","', encoding='latin-1')
df["Gestartet"] = pd.to_datetime(df["Gestartet"])

In [ ]:
df["Date"] = df["Gestartet"].dt.date
df2 = df2.rename(columns={"Zeitstempel":"Date", "Wert":"Temp_mittel"})

In [ ]:
df.set_index(pd.to_datetime(df['Date']), inplace=True)
df2.set_index(pd.to_datetime(df2['Date']), inplace=True)

In [ ]:
df_fertig = df.merge(df2, left_index=True, right_index=True, how='outer')

In [ ]:
df_fertig = df_fertig.dropna()

In [ ]:
df_fertig = df_fertig.reset_index()

In [ ]:
df_fertig["Jahr"] = df_fertig["Gestartet"].dt.strftime("%Y")
df_fertig["Monat"] = df_fertig["Gestartet"].dt.strftime("%m")
df_fertig["Wochentag"] = df_fertig["Gestartet"].dt.strftime("%A")

In [ ]:
df_fertig = df_fertig[['Gestartet','Verbrauch (kWh)','Temp_mittel','Jahr','Monat', 'Wochentag']]

In [ ]:
df_fertig["Ladesitzung"] = 1
df_fertig['Verbrauch (kWh)'] = df_fertig['Verbrauch (kWh)'].str.replace(',', '.').astype(float)
df_fertig = df_fertig[df_fertig['Verbrauch (kWh)'] > 0]

In [ ]:
data = df_fertig
data['Gestartet'] = pd.to_datetime(data['Gestartet'], format='%d.%m.%Y %H:%M')

df1 = data['Temp_mittel'].groupby([data['Gestartet'].dt.month_name(), data['Gestartet'].dt.year], sort=False).mean()
df2 = data["Verbrauch (kWh)"].groupby([data['Gestartet'].dt.month_name(), data['Gestartet'].dt.year], sort=False).mean()
temp = pd.DataFrame(df1)
verbrauch = pd.DataFrame(df2)

temp.index.rename(['month','year'], inplace=True) 
verbrauch.index.rename(['month','year'], inplace=True) 
temp.reset_index( inplace=True)
verbrauch.reset_index( inplace=True)

fig,ax = plt.subplots(figsize=(10, 5))

ax.plot(temp.index,temp. Temp_mittel, color="red", marker="o")
ax.set_xlabel("Monate (2018-2022)", fontsize = 14)
ax.set_ylabel(" \u00D8 Temperatur (°C)", color = "red",fontsize=14)

ticks = np.arange(0, 43, 1)
labels = ['December', '2019-January', 'February*', 'March*', 'April', 'May', 'June',
       'July*', 'August*', 'September*', 'October', 'November', 'December',
       '2020-January', 'February*', 'March*', 'April', 'May', 'June', 'July*',
       'August*', 'September*', 'October', 'November', 'December',
       '2021-January*', 'February*', 'March', 'April', 'May', 'June', 'July*',
       'August*', 'September', 'October', 'November', 'December',
       '2022-January', 'February*', 'March', 'April', 'May', 'June']

plt.xticks(ticks,labels)
plt.xticks(rotation=90)
ax2 = ax.twinx()
ax2.plot(temp.index , verbrauch["Verbrauch (kWh)"] , color="blue", marker="o")
ax2.set_ylabel(" \u00D8 Verbrauch (kWh)",color = "blue", fontsize=15) 
ax2.set_ylim(ymin = 0)
ax.grid(b = True, which='major', linestyle = '-')

plt.show()

# fig.savefig("Temperatur_Verbrauch")
